 NaverBlogCrawler 클래스의 주요 함수:

1. parse_blog_data(self, data):

    HTML 데이터를 파싱하여 블로그 제목, URL, 설명, 작성 날짜를 추출하는 함수입니다.
    BeautifulSoup를 사용하여 HTML 데이터를 구문 분석하고 원하는 정보를 추출합니다.

2. async_requests(self, url_lst):

    비동기 HTTP 요청을 사용하여 여러 블로그 페이지에서 데이터를 가져오는 함수입니다.
    aiohttp와 asyncio를 활용하여 비동기적으로 데이터를 수집합니다.

3. get_total_blog_count(self, data):

    검색 결과에서 전체 블로그 수를 파싱하는 함수입니다.
    검색 결과 데이터에서 블로그 수를 추출하여 반환합니다.

4. sync_request(self):

    동기적인 HTTP 요청을 사용하여 검색 결과에서 블로그 수를 가져오는 함수입니다.

5. crawl_blogs(self):

    주어진 페이지 범위 내에서 블로그 데이터를 크롤링하는 메인 함수입니다.
    비동기 방식으로 여러 페이지를 크롤링하고 데이터를 저장한 뒤 반환합니다.

In [2]:
# aiohttp 오류 발생 시 설치
!pip3 install aiohttp

In [6]:
import time
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd
import nest_asyncio

nest_asyncio.apply()

class NaverBlogCrawler:
    def __init__(self, keyword, start_page, end_page, start_date, end_date):
        self.keyword = keyword
        self.start_page = start_page
        self.end_page = end_page
        self.start_date = start_date
        self.end_date = end_date

        self.data = {}  # 크롤링한 데이터를 저장
        
    def parse_blog_data(self, data):
        soup = BeautifulSoup(data.replace("\\", ""), "html.parser")
        title_and_url_elems = soup.find_all("a", {"class": "api_txt_lines"})
        desc_elems = soup.find_all("div", {"class": "api_txt_lines"})
        date_elems = soup.find_all("span", {"class": "sub_time"})

        return [(title_and_url_elems[i].text, title_and_url_elems[i]['href'], desc_elems[i].text, date_elems[i].text)
                for i in range(len(title_and_url_elems))]

    def async_requests(self, url_lst):
        
        async def get_blog_data(_url):
            async with aiohttp.ClientSession() as session:
                async with session.get(_url) as response:
                    text = await response.text()
                    for title, url, desc, date in self.parse_blog_data(text):
                        self.data[url] = {'title': title, 'description': desc, 'date': date}

        loop = asyncio.get_event_loop()
        loop.run_until_complete(
            asyncio.gather(*(get_blog_data(url_lst[i]) for i in range(len(url_lst))))
        )

    def get_total_blog_count(self, data):
        data = data.replace('\\', '')
        start_index = data.find('"total":"') + 9
        end_index = data[start_index:start_index + 50].find('"') + start_index
        count = int(data[start_index:end_index])

        return count

    def sync_request(self):
        url = f'https://s.search.naver.com/p/blog/search.naver?where=blog&sm=tab_pge&api_type=1&query={self.keyword}&rev=44&start={int(self.start_page) * 30}&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so:dd,p:from{self.end_date}to{self.start_date}&nlu_query={{"r_category":"29+27"}}&dkey=0&source_query=&nx_search_query={self.keyword}&spq=0&_callback=viewMoreContents'
        response = requests.get(url)
        count = self.get_total_blog_count(response.text)
        return count

    def crawl_blogs(self):
        urls = [
            f'https://s.search.naver.com/p/blog/search.naver?where=blog&sm=tab_pge&api_type=1&query={self.keyword}&rev=44&start={i * 30}&dup_remove=1&post_blogurl=&post_blogurl_without=&nso=so:dd,p:from{self.end_date}to{self.start_date}&nlu_query={{"r_category":"29+27"}}&dkey=0&source_query=&nx_search_query={self.keyword}&spq=0&_callback=viewMoreContents'
            for i in range(self.start_page, self.end_page)
        ]
        self.async_requests(urls)
        return self.data

if __name__ == '__main__':
    
    naver_crawler = NaverBlogCrawler(keyword='고령화', start_page=1, end_page=130, start_date=20180101, end_date=20230901)
    crawled_data = naver_crawler.crawl_blogs()
    url_list = list(crawled_data.keys())
    df = pd.DataFrame.from_dict(crawled_data, orient='index').reset_index()
    df.columns = ['url', 'title', 'description', 'date']
    df['idx'] = df['url'].apply(lambda x: url_list.index(x)) #데이터 중복 방지
    df.to_csv('naver_blog_data.csv', index=False)

df

,url,title,description,date,idx
0,https://blog.naver.com/so-il/223199703877,『로컬에서 청년하다』,"그러나 다른 한편에서는 지역 소멸, 인구 소멸, 저출산·고령화라는 말이 유행처럼.....",2023.09.01.,0
1,https://blog.naver.com/pchshin/223199701348,"캐어유 X 케이제이이노베이션, 스마트경로당 키오스크...",최근 고령화 사회로 진입함에 따라 향후 지속적으로 성장할 것으로 예상되는 #스마트경...,2023.09.01.,1
2,https://blog.naver.com/ace_mt5/223199700607,명문제약: 신약 및 비만치료제 개발 등으로 보이는 미래 방향성?,"인구 고령화에 따른 의약품 수요에 발맞추어 사업을 확장해 왔습니다 또한, 이... ...",2023.09.01.,2
3,https://blog.naver.com/kimcecilia76/223199699803,뉴닉(NEWNEEK)_리뷰_20230901,이는 인구 고령화에 따른 사망률 증가와 비혼·만혼 증가에 따른 출산율 저하 등으로 ...,2023.09.01.,3
4,https://blog.naver.com/1qhtpbat/223199697993,문과 취업 잘되는 학과 순위는 과연?,"고령화 시대, 저출산의 시대에서 노인복지, 아동복지가 매우 중요해지는 시점입니다. ...",2023.09.01.,4
...,...,...,...,...,...
3823,https://blog.naver.com/mbcbpbp/223191574163,미용전문가란 무엇일까요~?,"같아요 고령화와 소득 향상, 그리고 삶의 질 추구 등 변화하는 사회적 관심과 케이(...",2023.08.23.,3823
3824,https://blog.naver.com/jarboy/223191572385,의료 수가 인상이 건강보험 '보장 강화'라는 윤석열 정부의...,"정부는 고령화 등 인구구조 변화로 인한 진료비 급증은 컨트롤하기 어렵다고 하지만, ...",2023.08.23.,3824
3825,https://blog.naver.com/normal332/223191568309,방통대 사회복지사 편입으로 2급 따기,인구의 고령화가 점점 가속화 되면서 노후에 대한 준비를 하기 위한 노력들이 지속되고...,2023.08.23.,3825
3826,https://blog.naver.com/coramdeo00000/223191567853,수원 직장인 사회복지사 준비생 실습 방법,최근 인구 고령화로 인해 독거노인 증가와 노인복지 및 다문화가정이 늘어남에 따라 취...,2023.08.23.,3826
